In [42]:
%load_ext autoreload
%autoreload 2

import time
import networkx as nx
import numpy as np
import logging
from copy import deepcopy
from collections import namedtuple
import torch
import pyro

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.parsing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from spatial_scene_grammars_examples.singles_pairs.grammar_constituency import *
#from spatial_scene_grammars_examples.singles_pairs.grammar_dependency import *
pyro.set_rng_seed(42)

grammar = SpatialSceneGrammar(
    root_node_type = Root,
    root_node_tf = torch.eye(4)
)
ground_truth_tree = grammar.sample_tree(detach=True)
observed_nodes = ground_truth_tree.get_observed_nodes()
print("Observed %d objects" % len(observed_nodes))
print("Objects: ", observed_nodes)

Observed 8 objects
Objects:  [<spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Root object at 0x7ff239d6c160>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff239f917f0>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a11ae48>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a224da0>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a224160>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a22f828>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a22f898>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff23a22f780>]


In [48]:
# Build up a big set of candidate intermediate nodes with both
# top-down and bottom-up generation.
proposed_poses_by_type = generate_candidate_node_pose_sets(
    grammar, observed_nodes, max_recursion_depth=10, verbose=True
)
print(proposed_poses_by_type)

17 unique bottom-up candidates, including observeds and root.
8 new top-down candidates.
{'Singles': [tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])], 'Pairs': [tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])], 'Pair': [tensor([[ 0.6432,  0.5753,  0.5053, -0.5814],
        [-0.0648,  0.6985, -0.7127,  0.7994],
        [-0.7629,  0.4257,  0.4866,  0.4884],
        [ 0.0000,  0.0000,  0.0000,  1.0000]]), tensor([[ 0.2497, -0.7531, -0.6086, -2.1240],
        [ 0.7928,  0.5199, -0.3180, -0.4837],
        [ 0.5559, -0.4031,  0.7269, -0.8704],
        [ 0.0000,  0.0000,  0.0000,  1.0000]]), tensor([[ 0.5430,  0.6174,  0.5692,  0.7865],
        [-0.3730, -0.4299,  0.8222,  0.4707],
        [ 0.7523, -0.6588, -0.0032, -1.0653],
        [ 0.0000,  0.0000,  0.0000,  1.0000]]), tensor([[ 0.7950, -0.6032,  0.0646,  0.7119],
        [ 0.2142,  0.3787,  0.9004, -1.3083],
        [-0.567

In [69]:
parse_trees = infer_mle_tree_with_mip_from_proposals(
    grammar, observed_nodes, proposed_poses_by_type, verbose=True, N_solutions=1
)

Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Thu Dec  2 18:01:41 2021

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 207 rows, 266 columns and 951 nonzeros
Model fingerprint: 0x3b1b1f55
Variable types: 0 continuous, 266 integer (266 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [4e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 110 rows and 151 columns
Presolve time: 0.01s
Presolved: 97 rows, 115 columns, 404 nonzeros
Variable types: 0 continuous, 115 integer (115 binary)
Found heuristic solution: objective 15.1107138

Root relaxation: objective -1.764696e+01, 46 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -17.64696    0    6   15.11071  -17.64696   217%     -    0s
H    0     0           